### This script needs following files: 
#### county_data.csv, Oxygen_1p.csv, tb_1p.csv, temp_1p.csv, ph_1p.csv

In [1]:
import pandas as pd
import numpy as np
import pickle
from geopy.geocoders import Nominatim

### 1. Find county for every record of water quality

In [2]:
geolocator = Nominatim()

In [3]:
def f(x):
    """
    This function finds county name for all coordinates, using with dataframe.apply() in later scripts
    x: pandas dataframe of four water quality indicators
    output: county name for each coordinater
    """
    la = x['Latitude']
    lo = x['Longitude']
    loc_marker = str(la)+str(lo)
    
    if loc_marker in loc_dict.keys():
        output = loc_dict[loc_marker]

    else:   
        str1 = str(la) + ', ' + str(lo)
        location = geolocator.reverse(str1)
        if 'county' in location.raw['address'].keys():
            c = location.raw['address']['county']
            c2 = c.encode('ascii', 'ignore')
            loc_dict[loc_marker]=c2
            if len(loc_dict) % 10 == 0:
                print len(loc_dict)
            output = c2
        else:
            output = 'no county'
        # print output

    return output

In [4]:
with open("loc_dict_4.pickle", "rb") as handle: # this pickle file contains the mapping from coordinates to counties generated by using geopy's API
    loc_dict = pickle.load(handle)

#### load data(truncated to one decimal)

In [5]:
oxy_1p = pd.read_csv('Oxygen_1p.csv')
ph_1p = pd.read_csv('ph_1p.csv')
temp_1p = pd.read_csv('temp_1p.csv')
tb_1p = pd.read_csv('tb_1p.csv')

#### find county name for every row

In [ ]:
oxy_1p['county'] = oxy_1p.apply(f, axis=1)
ph_1p['county'] = ph_1p.apply(f, axis=1)
tb_1p['county'] = tb_1p.apply(f, axis=1)
temp_1p['county'] = temp_1p.apply(f, axis=1)

#### save updated data to file

In [ ]:
oxy_1p.to_csv('oxy_1p_county.csv')
ph_1p.to_csv('ph_1p_county.csv')
tb_1p.to_csv('tb_1p_county.csv')
temp_1p.to_csv('temp_1p_county.csv')

### 2. Clean and group water quality data in every county

In [ ]:
# load county data
county = pd.read_csv('county_data.csv')
s = county['county'].unique()
county_list = list(s)

In [ ]:
def rchop(string, ending):
    """
    This function removes a specific substring from a given string
    string: a given string
    ending: the target substring to remove
    
    output: 
    string: updated string
    """
    if string.endswith(ending):
        return string[:-len(ending)]
    return string

#### clean and group all oxygen data in every county

In [ ]:
oxy = pd.read_csv('oxy_1p_county.csv')
oxy['county'] = oxy['county'].map(lambda x: rchop(x, ' County'))
oxy2 = oxy[oxy.county.isin(county_list)]
oxy3 = oxy2[['Result', 'county', 'Year']]
oxy_median_county_year = oxy3.groupby(['county', 'Year']).median().add_prefix('median_oxy_')
oxy_c = oxy2[['Result', 'county']]
oxy_median_county = oxy_c.groupby(['county']).median().add_prefix('median_oxy_').reset_index()
oxy_median_county_year.to_csv('oxy_median_county_year.csv')
oxy_median_county.to_csv('oxy_median_county.csv')

#### clean and group all pH data in every county

In [ ]:
ph = pd.read_csv('ph_1p_county.csv')
ph['county'] = ph['county'].map(lambda x: rchop(x, ' County'))
ph2 = ph[ph.county.isin(county_list)]
ph3 = ph2[['Result', 'county', 'Year']]
ph_median_county_year = ph3.groupby(['county', 'Year']).median().add_prefix('median_ph_')
ph_c = ph2[['Result', 'county']]
ph_median_county = ph_c.groupby(['county']).median().add_prefix('median_ph_').reset_index()
ph_median_county_year.to_csv('ph_median_county_year.csv')
ph_median_county.to_csv('ph_median_county.csv')

#### clean and group all temperature data in every county

In [ ]:
temp = pd.read_csv('temp_1p_county.csv')
temp['county'] = temp['county'].map(lambda x: rchop(x, ' County'))
temp2 = temp[temp.county.isin(county_list)]
temp3 = temp2[['Result', 'county', 'Year']]
temp_median_county_year = temp3.groupby(['county', 'Year']).median().add_prefix('median_temp_')
temp_c = temp2[['Result', 'county']]
temp_median_county = temp_c.groupby(['county']).median().add_prefix('median_temp_').reset_index()
temp_median_county_year.to_csv('temp_median_county_year.csv')
temp_median_county.to_csv('temp_median_county.csv')

#### clean and group all turbidity data in every county

In [ ]:
tb = pd.read_csv('tb_1p_county.csv')
tb['county'] = tb['county'].map(lambda x: rchop(x, ' County'))
tb2 = tb[tb.county.isin(county_list)]
tb3 = tb2[['Result', 'county', 'Year']]
tb_median_county_year = tb3.groupby(['county', 'Year']).median().add_prefix('median_tb_')
tb_c = tb2[['Result', 'county']]
tb_median_county = tb_c.groupby(['county']).median().add_prefix('median_tb_').reset_index()
tb_median_county_year.to_csv('tb_median_county_year.csv')
tb_median_county.to_csv('tb_median_county.csv')

### 3. Merge all water quality data to one file

In [ ]:
county_data = pd.read_csv('county_data.csv')
county_popden = county_data[['county', 'popu_density']]

In [ ]:
oxy_median_county = pd.read_csv('oxy_median_county.csv')
ph_median_county = pd.read_csv('ph_median_county.csv')
tb_median_county = pd.read_csv('tb_median_county.csv')
temp_median_county = pd.read_csv('temp_median_county.csv')

#### merger water quality data files one by one

In [ ]:
pop_oxy = pd.merge(county_popden, oxy_median_county, on='county', how='outer')
pop_oxy_tb = pd.merge(pop_oxy, tb_median_county, on='county', how='outer')
pop_oxy_tb_ph = pd.merge(pop_oxy_tb, ph_median_county, on='county', how='outer')
pop_oxy_tb_ph_temp = pd.merge(pop_oxy_tb_ph, temp_median_county, on='county', how='outer')
county_full = pop_oxy_tb_ph_temp[['county', 'popu_density', 'median_oxy_Result', 'median_tb_Result', 'median_ph_Result', 'median_temp_Result']]

In [ ]:
county_full.to_csv('county_full.csv')

In [ ]:
county_full